### Evaluator-Optimizer

Effective when we have clear evaluation criteria and when iterative refinement provides measurable value

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from langchain_groq import ChatGroq
llm = ChatGroq(model="openai/gpt-oss-20b", temperature = 0)

c:\dev\AgenticAI_Material\langgraph_tut\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing_extensions import Literal
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

In [3]:
#Graph State
class State(TypedDict):
    joke: str
    topic: str
    feedback: str
    funny_or_not: str
    

In [4]:
#Schema for feedback
class Feedback(TypedDict):
    grade: Literal["funny", "not_funny"] = Field(description="Decide if the joke if funny or not")
    feedback: str = Field(description="Feedback to modify the joke to make it funnier")

In [5]:
evaluator = llm.with_structured_output(Feedback)
evaluator

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001307CECF290>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001307CF9F310>, model_name='openai/gpt-oss-20b', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Feedback', 'description': "dict() -> new empty dictionary\ndict(mapping) -> new dictionary initialized from a mapping object's\n    (key, value) pairs\ndict(iterable) -> new dictionary initialized as if via:\n    d = {}\n    for k, v in iterable:\n        d[k] = v\ndict(**kwargs) -> new dictionary initialized with the name=value pairs\n    in the keyword argument li

In [ ]:
#Create nodes 

# Generate joke (check for incoming feedback first)
def generator(state:State):
    """LLM generates a joke"""
    
    if state.get('feedback'):
        msg = llm.invoke(
            [
                HumanMessage(content = f"Generate a joke on topic: {state['topic']} but take account of the feedback to improve it: {state['feedback']}")
            ]
        )
        return {"joke": msg.content}
    else:
        msg = llm.invoke(
            [
                HumanMessage(content = f"Generate a joke on the topic: {state['topic']}")
            ]
        )
        return {"joke": msg.content}

# Evaluate joke
def evaluate_joke(state: State):
    """LLM evaluates a joke"""
    grade = evaluator.invoke(f"Grade the joke {state['joke']}")
    return {"funny_or_not": grade.grade, 
            "feedback": grade.feedback}

#Conditional logic to send joke back to evaluator
def route_joke(state:State):
    """Route the joke back to the generator node depending on whether 
    it needs improvement, or end based on feedback.
    """
    if state['funny_or_not']=='funny':
        return "Accepted"
    elif state['funny_or_not']=="not_funny":
        return "Rejected + Feedback"
    

In [16]:
#Create the graph now
optimizer = StateGraph(State)

optimizer.add_node("generator", generator)
optimizer.add_node("evaluator", evaluate_joke)

optimizer.add_edge(START, "generator")
optimizer.add_edge("generator", "evaluator")
optimizer.add_conditional_edges('evaluator',
                                route_joke,
                                {'Accepted': END,
                                 'Rejected + Feedback': "generator"})

graph = optimizer.compile()

In [17]:
# Invoke
state = graph.invoke({"topic": "Cats"})
print(state["joke"])

AttributeError: 'function' object has no attribute 'invoke'